In [1]:
import cellex
import numpy as np # needed for formatting data for this tutorial
import pandas as pd # needed for formatting data for this tutorial
import requests

# Import and prepare data

#### Expression data

In [31]:
data = pd.read_csv('/home/cbmr/kzd307/gitte/hippocampus/data/Saunders.csv')

In [32]:
data.rename(columns={'Unnamed: 0':'bla'}, inplace=True )
data = data.set_index('bla')
data.index.name = None

In [33]:
data.shape

(22245, 52846)

In [34]:
data.head()

,P60HippoRep1P1_CTACGCGCATGN,P60HippoRep1P1_GCGTGGCGCGTA,P60HippoRep1P1_GTGCAGACGCAG,P60HippoRep1P1_CGATAAATGCAT,P60HippoRep1P1_CTGGCGAAGAAC,P60HippoRep1P1_GGTACAGAGGCG,P60HippoRep1P1_GCTCCTGGACTT,P60HippoRep1P1_ATACCTGCCGAG,P60HippoRep1P1_GACTGATGAACG,P60HippoRep1P1_CTGCGCGCATGN,...,P60HippoRep6P3_GCCTAATAGAGG,P60HippoRep6P3_GGTCTGGTCCCA,P60HippoRep6P3_AGGTTTCTCTCC,P60HippoRep6P3_ATCGCACAGTCT,P60HippoRep6P3_CCAGCACCACTC,P60HippoRep6P3_TATGTCCGAGCT,P60HippoRep6P3_AGTCGCTCGGTT,P60HippoRep6P3_TTTTCTCCCCGT,P60HippoRep6P3_GATTTCCCATAN,P60HippoRep6P3_TTCGCTAATATT
0610005C13RIK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610007P14RIK,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,4,0,0,0,2
0610009B22RIK,0,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009E02RIK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610009L18RIK,2,1,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


#### Metadata

In [42]:
metadata = pd.read_csv('/home/cbmr/kzd307/gitte/hippocampus/data/Saunders_subcluster_annotations.csv')

In [44]:
metadata.rename(columns={'Unnamed: 0':'bla','V2':'subcluster'}, inplace=True )
metadata = metadata.set_index('V1')
metadata.index.name = None
metadata = metadata.drop(["bla"],axis=1)

,subcluster
P60HippoRep1P1_CTACGCGCATGN,96
P60HippoRep1P1_GCGTGGCGCGTA,72
P60HippoRep1P1_GTGCAGACGCAG,69
P60HippoRep1P1_CGATAAATGCAT,69
P60HippoRep1P1_CTGGCGAAGAAC,69


In [45]:
metadata.shape

(52846, 1)

In [46]:
metadata.head()

,subcluster
P60HippoRep1P1_CTACGCGCATGN,96
P60HippoRep1P1_GCGTGGCGCGTA,72
P60HippoRep1P1_GTGCAGACGCAG,69
P60HippoRep1P1_CGATAAATGCAT,69
P60HippoRep1P1_CTGGCGAAGAAC,69


___

### Convert gene ID's to human

Target number of genes is 22245

In [47]:
# Convert genes from mouse to human
r = requests.post(
    url='https://biit.cs.ut.ee/gprofiler/api/orth/orth/',
    json={
        'organism':'mmusculus',
        'target':'hsapiens',
        'query':data.index.tolist(),
    }
    )

In [48]:
human_id = pd.DataFrame(r.json()['result'])

In [49]:
filtered_ids = human_id[human_id["n_result"]==1]
filtered_id_index = human_id[human_id["n_result"]==1].index.tolist()
filtered_ids.shape

(23442, 11)

In [50]:
filtered_ids2 = filtered_ids[filtered_ids["n_converted"]==1]
filtered_id_index2 = filtered_ids[filtered_ids["n_converted"]==1].index.tolist()
filtered_ids2.shape

(22244, 11)

____________________________________


#### Find missing gene

In [13]:
difference = list(set(data.index.tolist()).symmetric_difference(set(filtered_ids2["incoming"].tolist())))
list_difference

['GM1123']

In [15]:
'GM1123' in data.index.tolist()

True

In [16]:
'GM1123' in filtered_ids2["incoming"]

False

___

#### Prepare four dataframes:
mouse_data (containing mouse gene ID's)
<br>
human_data (containing human gene ID's)
<br>
name_data (containing gene name)
<br>
original_data (containing original ID's)

In [51]:
# Merge expression data with ID conversions
merged_left = pd.merge(left=filtered_ids2, right=data, how='left', left_on='incoming', right_on=data.index)
merged_left.head()

,converted,description,incoming,n_converted,n_incoming,name,namespaces,disambiguate,ortholog_ensg,n_result,...,P60HippoRep6P3_GCCTAATAGAGG,P60HippoRep6P3_GGTCTGGTCCCA,P60HippoRep6P3_AGGTTTCTCTCC,P60HippoRep6P3_ATCGCACAGTCT,P60HippoRep6P3_CCAGCACCACTC,P60HippoRep6P3_TATGTCCGAGCT,P60HippoRep6P3_AGTCGCTCGGTT,P60HippoRep6P3_TTTTCTCCCCGT,P60HippoRep6P3_GATTTCCCATAN,P60HippoRep6P3_TTCGCTAATATT
0,ENSMUSG00000109644,N/A,0610005C13RIK,1,1,N/A,MGI,True,N/A,1,...,0,0,0,0,0,0,0,0,0,0
1,N/A,N/A,0610007P14RIK,1,2,N/A,,False,N/A,1,...,0,0,0,0,0,4,0,0,0,2
2,ENSMUSG00000007777,N/A,0610009B22RIK,1,3,N/A,"ENTREZGENE,MGI,UNIPROT_GN,WIKIGENE",True,N/A,1,...,0,0,0,0,0,0,0,0,0,0
3,ENSMUSG00000086714,N/A,0610009E02RIK,1,4,N/A,MGI,True,N/A,1,...,0,0,0,0,0,0,0,0,0,0
4,ENSMUSG00000043644,N/A,0610009L18RIK,1,5,N/A,MGI,True,N/A,1,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Remove genes with no ID
remove_NA = merged_left[merged_left['ortholog_ensg']=='N/A'].index
data_full = merged_left.drop(index=remove_NA)

##### Mouse data

In [53]:
mouse_data = data_full.drop(['description', 'incoming','n_converted','n_incoming','name','namespaces','disambiguate','ortholog_ensg','n_result','query'], axis=1)

mouse_data = mouse_data.set_index('converted')
mouse_data.index.name = None

mouse_data.head()

,P60HippoRep1P1_CTACGCGCATGN,P60HippoRep1P1_GCGTGGCGCGTA,P60HippoRep1P1_GTGCAGACGCAG,P60HippoRep1P1_CGATAAATGCAT,P60HippoRep1P1_CTGGCGAAGAAC,P60HippoRep1P1_GGTACAGAGGCG,P60HippoRep1P1_GCTCCTGGACTT,P60HippoRep1P1_ATACCTGCCGAG,P60HippoRep1P1_GACTGATGAACG,P60HippoRep1P1_CTGCGCGCATGN,...,P60HippoRep6P3_GCCTAATAGAGG,P60HippoRep6P3_GGTCTGGTCCCA,P60HippoRep6P3_AGGTTTCTCTCC,P60HippoRep6P3_ATCGCACAGTCT,P60HippoRep6P3_CCAGCACCACTC,P60HippoRep6P3_TATGTCCGAGCT,P60HippoRep6P3_AGTCGCTCGGTT,P60HippoRep6P3_TTTTCTCCCCGT,P60HippoRep6P3_GATTTCCCATAN,P60HippoRep6P3_TTCGCTAATATT
ENSMUSG00000042208,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000020831,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000058706,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
ENSMUSG00000060512,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSMUSG00000090066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Human data

In [54]:
human_data = data_full.drop(['description', 'incoming','n_converted','n_incoming','name','namespaces','disambiguate','n_result','query','converted'], axis=1)

human_data = human_data.set_index('ortholog_ensg')
human_data.index.name = None

human_data.head()

,P60HippoRep1P1_CTACGCGCATGN,P60HippoRep1P1_GCGTGGCGCGTA,P60HippoRep1P1_GTGCAGACGCAG,P60HippoRep1P1_CGATAAATGCAT,P60HippoRep1P1_CTGGCGAAGAAC,P60HippoRep1P1_GGTACAGAGGCG,P60HippoRep1P1_GCTCCTGGACTT,P60HippoRep1P1_ATACCTGCCGAG,P60HippoRep1P1_GACTGATGAACG,P60HippoRep1P1_CTGCGCGCATGN,...,P60HippoRep6P3_GCCTAATAGAGG,P60HippoRep6P3_GGTCTGGTCCCA,P60HippoRep6P3_AGGTTTCTCTCC,P60HippoRep6P3_ATCGCACAGTCT,P60HippoRep6P3_CCAGCACCACTC,P60HippoRep6P3_TATGTCCGAGCT,P60HippoRep6P3_AGTCGCTCGGTT,P60HippoRep6P3_TTTTCTCCCCGT,P60HippoRep6P3_GATTTCCCATAN,P60HippoRep6P3_TTCGCTAATATT
ENSG00000162929,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
ENSG00000161939,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000168887,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
ENSG00000154274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000248713,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Name data

In [55]:
name_data = data_full.drop(['description', 'incoming','n_converted','n_incoming','namespaces','disambiguate','n_result','query','converted','ortholog_ensg'], axis=1)

name_data = name_data.set_index('name')
name_data.index.name = None

name_data.head()

,P60HippoRep1P1_CTACGCGCATGN,P60HippoRep1P1_GCGTGGCGCGTA,P60HippoRep1P1_GTGCAGACGCAG,P60HippoRep1P1_CGATAAATGCAT,P60HippoRep1P1_CTGGCGAAGAAC,P60HippoRep1P1_GGTACAGAGGCG,P60HippoRep1P1_GCTCCTGGACTT,P60HippoRep1P1_ATACCTGCCGAG,P60HippoRep1P1_GACTGATGAACG,P60HippoRep1P1_CTGCGCGCATGN,...,P60HippoRep6P3_GCCTAATAGAGG,P60HippoRep6P3_GGTCTGGTCCCA,P60HippoRep6P3_AGGTTTCTCTCC,P60HippoRep6P3_ATCGCACAGTCT,P60HippoRep6P3_CCAGCACCACTC,P60HippoRep6P3_TATGTCCGAGCT,P60HippoRep6P3_AGTCGCTCGGTT,P60HippoRep6P3_TTTTCTCCCCGT,P60HippoRep6P3_GATTTCCCATAN,P60HippoRep6P3_TTCGCTAATATT
KIAA1841,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
RNASEK-C17orf49,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C2orf68,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
C4orf19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
C4orf54,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Original data

In [56]:
original_data = data_full.drop(['description','n_converted','n_incoming','namespaces','disambiguate','n_result','query','converted','ortholog_ensg','name'], axis=1)

original_data = original_data.set_index('incoming')
original_data.index.name = None

original_data.head()

,P60HippoRep1P1_CTACGCGCATGN,P60HippoRep1P1_GCGTGGCGCGTA,P60HippoRep1P1_GTGCAGACGCAG,P60HippoRep1P1_CGATAAATGCAT,P60HippoRep1P1_CTGGCGAAGAAC,P60HippoRep1P1_GGTACAGAGGCG,P60HippoRep1P1_GCTCCTGGACTT,P60HippoRep1P1_ATACCTGCCGAG,P60HippoRep1P1_GACTGATGAACG,P60HippoRep1P1_CTGCGCGCATGN,...,P60HippoRep6P3_GCCTAATAGAGG,P60HippoRep6P3_GGTCTGGTCCCA,P60HippoRep6P3_AGGTTTCTCTCC,P60HippoRep6P3_ATCGCACAGTCT,P60HippoRep6P3_CCAGCACCACTC,P60HippoRep6P3_TATGTCCGAGCT,P60HippoRep6P3_AGTCGCTCGGTT,P60HippoRep6P3_TTTTCTCCCCGT,P60HippoRep6P3_GATTTCCCATAN,P60HippoRep6P3_TTCGCTAATATT
0610010F05RIK,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
0610010K14RIK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610030E20RIK,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
0610040J01RIK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1110002E22RIK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


___
## Create ESObject and compute expression specificity

In [57]:
eso = cellex.ESObject(data=human_data, annotation=metadata, verbose=True)

Preprocessing - checking input ... input parsed in 0 min 0 sec
Preprocessing - running remove_non_expressed ... excluded 0 / 15474 genes in 0 min 15 sec
Preprocessing - normalizing data ... data normalized in 0 min 17 sec
Preprocessing - running ANOVA ... excluded 1222 / 15474 genes in 0 min 48 sec


In [58]:
eso.compute(verbose=True)

Computing DET ... 
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 45 sec
Computing EP ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 0 sec
Computing GES ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 10 sec
Computing NSI ...
    esw ...
    empirical p-values ...
    esw_s ...
    finished in 0 min 31 sec
Computing ESmu ...
    finished in 0 min 0 sec
Computing ESsd ...
    finished in 0 min 0 sec
Computed ['det.esw', 'det.esw_null', 'det.pvals', 'det.esw_s', 'ep.esw', 'ep.esw_null', 'ep.pvals', 'ep.esw_s', 'ges.esw', 'ges.esw_null', 'ges.pvals', 'ges.esw_s', 'nsi.esw', 'nsi.esw_null', 'nsi.pvals', 'nsi.esw_s', 'esmu', 'essd'].


In [59]:
eso.save_as_csv(path='/home/cbmr/kzd307/gitte/hippocampus/data/saunders_cellex_subcluster', file_prefix='hippocampus_mouse_cells_subclusters', verbose=True)


Saving results as csv to disk ...
  Saved: /home/cbmr/kzd307/gitte/hippocampus/data/saunders_cellex_subcluster/hippocampus_mouse_cells_subclusters.esmu.csv.gz
  Saved: /home/cbmr/kzd307/gitte/hippocampus/data/saunders_cellex_subcluster/hippocampus_mouse_cells_subclusters.essd.csv.gz
Finished saving results to /home/cbmr/kzd307/gitte/hippocampus/data/saunders_cellex_subcluster


In [60]:
eso.results["esmu"].head()

,10,100,101,102,103,104,107,108,11,110,...,86,88,89,9,90,91,93,94,96,99
gene,,,,,,,,,,,,,,,,,,,,,
ENSG00000162929,0.0,0.033699,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
ENSG00000168887,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.077054,0.0,0.0,...,0.0,0.089398,0.000000,0.0,0.000000,0.00000,0.050404,0.0,0.000000,0.000000
ENSG00000154274,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.869036,0.0,0.944591,0.95184,0.801056,0.0,0.494365,0.000000
ENSG00000248713,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.000000
ENSG00000110696,0.0,0.290076,0.049085,0.194689,0.160029,0.285086,0.0,0.000000,0.0,0.0,...,0.0,0.129566,0.203100,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.046226


In [61]:
eso.results["essd"].head()

,10,100,101,102,103,104,107,108,11,110,...,86,88,89,9,90,91,93,94,96,99
gene,,,,,,,,,,,,,,,,,,,,,
ENSG00000162929,0.0,0.058369,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.0,0.000000,0.000000
ENSG00000168887,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.086787,0.0,0.0,...,0.0,0.057277,0.000000,0.0,0.0000,0.000000,0.072739,0.0,0.000000,0.000000
ENSG00000154274,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.062391,0.0,0.0251,0.029425,0.058692,0.0,0.085326,0.000000
ENSG00000248713,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.0,0.000000,0.000000
ENSG00000110696,0.0,0.265854,0.085017,0.270832,0.259912,0.297419,0.0,0.000000,0.0,0.0,...,0.0,0.200258,0.247150,0.0,0.0000,0.000000,0.000000,0.0,0.000000,0.080065
